# ResNet-50

In this file, I fine-tuned ResNet50 deep neural network to recognize all the "hand" images detected by MaskRCNN, I train the model on 433 data, and I save the weight of the model at last for future use.

In [25]:
from __future__ import division, print_function, absolute_import

import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

from keras.applications.resnet50 import ResNet50
from keras.backend.tensorflow_backend import set_session
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.utils.np_utils import to_categorical
from keras.layers import Input
from keras.layers import Dense, Flatten, Dropout
from keras.initializers import RandomNormal
from keras.models import Model
from keras import backend as K
from keras.models import load_model
from keras import metrics
from keras.models import model_from_json
from sklearn.cross_validation import train_test_split

In [2]:
# Train dataset path
data_path = './Training_Set'
train_list = os.path.join(data_path, 'train.list')

In [26]:
# Clear session
K.clear_session()

In [27]:
# Load pre-trained resnet50
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten(name='flatten')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dropout(0.5)(x)
x = Dense(11, activation='softmax', name='predictions')(x)
net = Model(input=base_model.input, output=x)

d:\programs\anaconda\envs\python35\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("pr..., inputs=Tensor("in...)`
  if sys.path[0] == '':


In [28]:
net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [29]:
net.compile(loss="categorical_crossentropy",
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [30]:
# Load data
images, labels = [], []

file = open(train_list)
for text in file.readlines():
    name = text.strip().split("-")[0]
    lbl = text.strip().split("-")[1]
    labels.append(lbl)
    img = Image.open(os.path.join(data_path, name)).resize((224,224))
    img = np.array(img)
    images.append(img)
        
images = np.array(images)
images = images.astype('float32')
images -= np.mean(images)
images /= np.max(images)
labels = to_categorical(labels)

In [31]:
# # Train
# batch_size = 128
# datagen = ImageDataGenerator(rescale=1./255, 
#                              shear_range=0.2, 
#                              zoom_range=0.2, 
#                              horizontal_flip=True)

# generator = datagen.flow(images, labels, batch_size=batch_size)

# net.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# net.fit_generator(generator, epochs=100)

In [32]:
# Split training data and test data
Xtrain, Xtest, Ytrain, Ytest = train_test_split(images, labels, test_size=0.1)

In [33]:
# Train model
batch_size = 128
net.fit(Xtrain, Ytrain, 
          validation_data=(Xtest, Ytest),
          batch_size=batch_size,
          epochs=100,
          shuffle=True)

Train on 389 samples, validate on 44 samples
Epoch 1/100
389/389 [==============================] - 3s 7ms/step - loss: 3.3883 - acc: 0.0951 - val_loss: 2.7118 - val_acc: 0.1364
Epoch 2/100
389/389 [==============================] - 2s 6ms/step - loss: 2.5811 - acc: 0.1902 - val_loss: 2.8123 - val_acc: 0.0909
Epoch 3/100
389/389 [==============================] - 2s 6ms/step - loss: 2.4076 - acc: 0.2185 - val_loss: 2.9928 - val_acc: 0.0682
Epoch 4/100
389/389 [==============================] - 2s 6ms/step - loss: 2.1097 - acc: 0.3059 - val_loss: 2.8402 - val_acc: 0.0909
Epoch 5/100
389/389 [==============================] - 2s 6ms/step - loss: 1.8470 - acc: 0.3830 - val_loss: 2.8466 - val_acc: 0.1364
Epoch 6/100
389/389 [==============================] - 2s 6ms/step - loss: 1.6930 - acc: 0.4242 - val_loss: 3.0176 - val_acc: 0.0682
Epoch 7/100
389/389 [==============================] - 2s 6ms/step - loss: 1.7363 - acc: 0.4113 - val_loss: 3.0137 - val_acc: 0.0682
Epoch 8/100
389/389 [===

Epoch 62/100
389/389 [==============================] - 2s 6ms/step - loss: 0.6134 - acc: 0.7943 - val_loss: 3.4616 - val_acc: 0.0682
Epoch 63/100
389/389 [==============================] - 2s 6ms/step - loss: 0.5831 - acc: 0.7969 - val_loss: 3.4068 - val_acc: 0.0909
Epoch 64/100
389/389 [==============================] - 2s 6ms/step - loss: 0.4769 - acc: 0.8509 - val_loss: 3.6411 - val_acc: 0.0682
Epoch 65/100
389/389 [==============================] - 2s 6ms/step - loss: 0.5949 - acc: 0.7866 - val_loss: 3.8101 - val_acc: 0.0682
Epoch 66/100
389/389 [==============================] - 2s 6ms/step - loss: 0.5152 - acc: 0.8201 - val_loss: 3.9094 - val_acc: 0.0909
Epoch 67/100
389/389 [==============================] - 3s 6ms/step - loss: 0.6972 - acc: 0.7584 - val_loss: 3.6597 - val_acc: 0.0682
Epoch 68/100
389/389 [==============================] - 2s 6ms/step - loss: 0.6314 - acc: 0.7763 - val_loss: 3.1135 - val_acc: 0.0682
Epoch 69/100
389/389 [==============================] - 2s 6ms

In [35]:
# Train next 100 epoch if first 100 epoch didn't converge well
net.fit(Xtrain, Ytrain, 
          validation_data=(Xtest, Ytest),
          batch_size=batch_size,
          epochs=200,
          initial_epoch=100,
          shuffle=True)

Train on 389 samples, validate on 44 samples
Epoch 101/200
389/389 [==============================] - 3s 8ms/step - loss: 0.5237 - acc: 0.8406 - val_loss: 3.4709 - val_acc: 0.0682
Epoch 102/200
389/389 [==============================] - 2s 6ms/step - loss: 0.4279 - acc: 0.8406 - val_loss: 3.4215 - val_acc: 0.0909
Epoch 103/200
389/389 [==============================] - 2s 6ms/step - loss: 0.4746 - acc: 0.8380 - val_loss: 3.2548 - val_acc: 0.0909
Epoch 104/200
389/389 [==============================] - 2s 6ms/step - loss: 0.4437 - acc: 0.8535 - val_loss: 3.0747 - val_acc: 0.0682
Epoch 105/200
389/389 [==============================] - 2s 6ms/step - loss: 0.4468 - acc: 0.8458 - val_loss: 3.1382 - val_acc: 0.0909
Epoch 106/200
389/389 [==============================] - 2s 6ms/step - loss: 0.4347 - acc: 0.8663 - val_loss: 3.1583 - val_acc: 0.0909
Epoch 107/200
389/389 [==============================] - 2s 6ms/step - loss: 0.3800 - acc: 0.8689 - val_loss: 3.0416 - val_acc: 0.0909
Epoch 108/

389/389 [==============================] - 2s 6ms/step - loss: 0.2621 - acc: 0.8972 - val_loss: 4.6208 - val_acc: 0.0909
Epoch 162/200
389/389 [==============================] - 2s 6ms/step - loss: 0.1979 - acc: 0.9357 - val_loss: 4.2777 - val_acc: 0.0909
Epoch 163/200
389/389 [==============================] - 2s 6ms/step - loss: 0.2763 - acc: 0.9049 - val_loss: 4.1410 - val_acc: 0.0909
Epoch 164/200
389/389 [==============================] - 2s 6ms/step - loss: 0.2128 - acc: 0.9332 - val_loss: 4.4593 - val_acc: 0.0682
Epoch 165/200
389/389 [==============================] - 2s 6ms/step - loss: 0.2788 - acc: 0.8997 - val_loss: 5.2965 - val_acc: 0.0682
Epoch 166/200
389/389 [==============================] - 2s 6ms/step - loss: 0.2812 - acc: 0.9023 - val_loss: 5.4730 - val_acc: 0.0682
Epoch 167/200
389/389 [==============================] - 2s 6ms/step - loss: 0.3720 - acc: 0.8843 - val_loss: 5.2940 - val_acc: 0.0682
Epoch 168/200
389/389 [==============================] - 2s 6ms/step 

In [36]:
# Save model weigth
# serialize model to JSON
model_json = net.to_json()
with open("./Weight/model_ResNet.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
net.save_weights("./Weight/model_ResNet.h5")